In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
sys.argv=['']
del sys
os.chdir("../")


In [2]:

import matplotlib.pyplot as plt
import json
import math
import torch
import numpy
import pandas
import logging
import numpy as np
from tslearn.metrics import dtw



from IPython import embed
from common import scikit_wrappers, data_preprocess 
from common.utils import print_to_json
from common.dataloader import load_SMD_dataset, load_CSV_dataset
from common.sliding import BatchSlidingWindow, WindowIterator
from common.config import parse_arguments, set_logger, initialize_config


args = parse_arguments()

%matplotlib inline

In [3]:
# load config
config_dir = "./hypers/" if not args["load"] else args["load"]
params = initialize_config(config_dir, args)

In [6]:
pp = data_preprocess.preprocessor()
data_dict1 = load_CSV_dataset(params["path"], dataset="[3]: 1583_295")
data_dict2 = load_CSV_dataset(params["path"], dataset="[0]: 15449476_6265666") 
data_dict1_disc = pp.discretize(data_dict1, mode="train")
data_dict2_disc = pp.discretize(data_dict2, mode="train")

In [7]:
train_windows1, test_windows1 = data_preprocess.generate_windows(data_dict1, window_size=params["window_size"])
train_windows2, test_windows2 = data_preprocess.generate_windows(data_dict2, window_size=params["window_size"])
train_windows1_disc, test_windows1_disc = data_preprocess.generate_windows(data_dict1_disc, window_size=params["window_size"])
train_windows2_disc, test_windows2_disc = data_preprocess.generate_windows(data_dict2_disc, window_size=params["window_size"])

In [8]:
data_dict = load_CSV_dataset(params["path"])
data_dict_disc = pp.discretize(data_dict, mode="train")

In [9]:
train_windows_disc, test_windows_disc = data_preprocess.generate_windows(data_dict_disc, window_size=params["window_size"])

In [45]:
min(test_windows_disc.cpu().numpy()[:,0,:].reshape(-1))

10.0

In [47]:
max(train_windows_disc.cpu().numpy()[:,0,:].reshape(-1))

999.0

In [14]:
seqs = train_windows_disc.cpu().numpy()[:,0,:]

In [24]:
seqs_str = [list(seq) for seq in seqs.astype(str)]

In [16]:
from gensim.models import Word2Vec

2020-11-12 16:45:15,941 P53769 INFO 'pattern' package not found; tag filters are not available for English


In [29]:
model = Word2Vec(sentences=seqs_str, size=100, window=params["window_size"], min_count=1, workers=4)

2020-11-12 16:51:24,908 P53769 INFO collecting all words and their counts
2020-11-12 16:51:24,909 P53769 INFO PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-11-12 16:51:24,979 P53769 INFO PROGRESS: at sentence #10000, processed 450000 words, keeping 434 word types
2020-11-12 16:51:25,048 P53769 INFO PROGRESS: at sentence #20000, processed 900000 words, keeping 540 word types
2020-11-12 16:51:25,115 P53769 INFO PROGRESS: at sentence #30000, processed 1350000 words, keeping 583 word types
2020-11-12 16:51:25,183 P53769 INFO PROGRESS: at sentence #40000, processed 1800000 words, keeping 609 word types
2020-11-12 16:51:25,248 P53769 INFO PROGRESS: at sentence #50000, processed 2250000 words, keeping 636 word types
2020-11-12 16:51:25,313 P53769 INFO collected 648 word types from a corpus of 2690100 raw words and 59780 sentences
2020-11-12 16:51:25,315 P53769 INFO Loading a fresh vocabulary
2020-11-12 16:51:25,316 P53769 INFO effective_min_count=1 retains 648 unique 

In [56]:
model.wv.most_similar("787.0")

[('999.0', 0.4464571475982666),
 ('749.0', 0.35341572761535645),
 ('73.0', 0.3297819197177887),
 ('659.0', 0.31378233432769775),
 ('124.0', 0.2960684895515442),
 ('408.0', 0.28490298986434937),
 ('195.0', 0.2616068720817566),
 ('142.0', 0.2444586157798767),
 ('409.0', 0.2392110526561737),
 ('125.0', 0.23146170377731323)]

In [59]:
model.wv.most_similar("174.0")

[('122.0', 0.39886119961738586),
 ('116.0', 0.3243066668510437),
 ('49.0', 0.3088116943836212),
 ('452.0', 0.3022875189781189),
 ('23.0', 0.2951423227787018),
 ('512.0', 0.27868640422821045),
 ('97.0', 0.27154213190078735),
 ('364.0', 0.27152130007743835),
 ('764.0', 0.2674259543418884),
 ('494.0', 0.26684510707855225)]

In [54]:
model.wv.vocab.keys()

dict_keys(['216.0', '213.0', '361.0', '290.0', '198.0', '252.0', '463.0', '217.0', '291.0', '284.0', '373.0', '235.0', '270.0', '334.0', '274.0', '132.0', '368.0', '366.0', '232.0', '295.0', '229.0', '241.0', '297.0', '279.0', '276.0', '299.0', '194.0', '289.0', '778.0', '304.0', '818.0', '358.0', '347.0', '268.0', '330.0', '205.0', '220.0', '379.0', '314.0', '360.0', '307.0', '349.0', '233.0', '275.0', '313.0', '339.0', '282.0', '256.0', '390.0', '257.0', '271.0', '301.0', '331.0', '318.0', '239.0', '329.0', '320.0', '185.0', '231.0', '316.0', '221.0', '345.0', '323.0', '389.0', '238.0', '251.0', '364.0', '172.0', '164.0', '124.0', '262.0', '416.0', '303.0', '400.0', '261.0', '298.0', '223.0', '189.0', '338.0', '225.0', '199.0', '321.0', '296.0', '328.0', '125.0', '383.0', '327.0', '247.0', '272.0', '188.0', '322.0', '209.0', '287.0', '184.0', '211.0', '337.0', '375.0', '305.0', '309.0', '219.0', '161.0', '340.0', '319.0', '317.0', '310.0', '266.0', '336.0', '410.0', '250.0', '240.0',

In [60]:
train_windows1_disc[4980]

tensor([[228., 327., 315., 401., 203., 324., 319., 207., 418., 250., 233., 348.,
         244., 291., 422., 216., 372., 403., 234., 358., 340., 840., 386., 266.,
         324., 341., 280., 288., 409., 221., 411., 213., 393., 292., 358., 247.,
         285., 185., 376., 393., 311., 411., 425., 274., 248.]])

In [61]:
train_windows2_disc[4980]

tensor([[267., 250., 238., 290., 211., 298., 151., 338., 261., 303., 320., 255.,
         179., 311., 218., 306., 213., 230., 298., 999., 348., 346., 281., 311.,
         185., 349., 315., 196., 272., 215., 257., 349., 255., 217., 354., 310.,
         349., 232., 248., 303., 227., 164., 282., 306., 301.]])

In [87]:
vec1 = get_sum_repr(train_windows1_disc[4980], model)
vec2 = get_sum_repr(train_windows2_disc[4980], model)

In [88]:
from sklearn.metrics.pairwise import cosine_similarity
cosine_similarity(vec1.reshape(1,-1), vec2.reshape(1,-1))

array([[0.9212485]], dtype=float32)

In [86]:
def get_sum_repr(ts, model):
    mat = np.array([model.wv[str(item)] for item in ts.cpu().numpy().reshape(-1)])
    mat = mat.max(0)
    return mat